## Understanding reservoir water levels using OPERA DSWx-HLS data

[Lake Mead](https://en.wikipedia.org/wiki/Lake_Mead) is a water reservoir in southwestern United States and is significant for irrigation in neighboring states. The lake has experienced significant drought over the past decade and particularly during 2020-2023. In this notebook we demonstrate visualizing time series data, along with a vector shape that outlines the nominal levels of the lake.

In [ ]:
from warnings import filterwarnings
filterwarnings("ignore") # suppress PySTAC warnings

import rasterio
import rioxarray
from rasterio.crs import CRS
from rasterio.warp import transform_bounds
import geoviews as gv
from geoviews import opts
import geopandas as gpd

import hvplot.xarray  # noqa

from shapely.geometry import Point
from osgeo import gdal
import xyzservices.providers as xyz

# STAC imports to retrieve cloud data
from pystac_client import Client

from datetime import datetime
import numpy as np

from util_functions import search_to_df, urls_to_dataset

from util_functions import filter_search_by_cc

# GDAL setup for accessing cloud data
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF, TIFF')

## Use shapefile to demo vector data

In [ ]:
lake_mead = (-114.348, 36.423)

ref_crs = CRS.from_epsg(4326)
dst_crs = CRS.from_epsg(3857)
map_bounds = transform_bounds(ref_crs, dst_crs, *Point(*lake_mead).buffer(2).bounds)

In [ ]:
lake_mead_gv = gv.Points([lake_mead])

basemap = gv.tile_sources.OSM
plot = (lake_mead_gv*basemap).opts(
    opts.Points(
        color='red',
        alpha=0.75,
        size=25,
        width=800,
        height=800,
        xlim=(map_bounds[0], map_bounds[2]),
        ylim=(map_bounds[1], map_bounds[3]))
)
plot

In [ ]:
%%time
# We will search the available product record
start_date = datetime(year=2023, month=3, day=1)
stop_date = datetime.now()
date_range = f'{start_date.strftime("%Y-%m-%d")}/{stop_date.strftime("%Y-%m-%d")}'

# We open a client instance to search for data, and retrieve relevant data records
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'

# Setup PySTAC client
# POCLOUD refers to the PO DAAC cloud environment that hosts earth observation data
catalog = Client.open(f'{STAC_URL}/POCLOUD/')
collections = ["OPERA_L3_DSWX-HLS_V1_1.0"]

# Setup search options
search_opts = {
    'bbox' : Point(*lake_mead).buffer(0.01).bounds, 
    'collections': collections,
    'datetime' : date_range,
}

# Execute the search
search = catalog.search(**search_opts)
results = list(search.items_as_dicts())
print(f"Number of tiles found intersecting given AOI: {len(results)}")

Note: This next step can take a long time.

In [ ]:
%%time
# let's filter our results so that only scenes with less than 10% cloud cover are returned
results = filter_search_by_cc(results)

print("Number of results containing less than 10% cloud cover: ", len(results))

In [ ]:
# Load results into dataframe
granules = search_to_df(results, layer_name='0_B01_WTR')

In [ ]:
# Let's filter by tile id so that we can study changes over the same spatial extent
granules = granules[granules.tile_id=='T11SQA']
display(granules.head())
print(granules.shape)

In [ ]:
# Similarly, loading a year's worth of data will take a few minutes
%time dataset= urls_to_dataset(granules)

In [ ]:
# Define a colormap
COLORS = [(255, 255, 255, 0.1)]*256 # setting all colors to transparent initially
COLORS[0] = (0, 255, 0, 0.1) # Setting not water class to green
COLORS[1] = (0, 0, 255, 1) # Open surface water
COLORS[2] = (0, 0, 255, 1) # Partial surface water

In [ ]:
# Let's load the vector shapefile of the Lake

gdf = gpd.read_file('shapefiles/lake_mead_shapefiles/lakebnds.shp')

In [ ]:
gdf.head()

The shapefile contains a lot of features, but we're interested only in the overall outline of the reservoir. We can use the `dissolve` method to combine the geometry column into a single `Polygon`

In [ ]:
gdf_dissolved = gdf.dissolve() # dissove all geometries into a single polygon
gdf_dissolved.plot(); # geopandas has inbuilt methods to plot the geometry column

In [ ]:
gdf_dissolved.to_crs(dst_crs, inplace=True) # project shape to web mercator to help with plotting

In [ ]:
lake_mead_usgs_shape = gv.Polygons(gdf)

In [ ]:
basemap = gv.tile_sources.OSM
plot = (lake_mead_usgs_shape*basemap).opts(
    opts.Polygons(
        color='red',
        alpha=0.5,
        width=900,
        height=900,
))
plot

In [ ]:
img = dataset.hvplot.image(title = 'Lake Mead, NV USA - water extent over a year',
                            x='lon', y='lat', 
                            project=True, rasterize=False,
                            framewise=False, 
                            cmap=COLORS, 
                            colorbar=False,
                            widget_location='bottom',
                            tiles = gv.tile_sources.ESRI,
                            xlabel='Longitude (degrees)',ylabel='Latitude (degrees)',
                            fontscale=1.25,
                            frame_width=1000, frame_height=1000,)

img